In [1]:
# Installing required packages
!pip install langchainhub
!pip install langchain-openai
!pip install langchain
!pip install beautifulsoup4
!pip install langchain-community
!pip install faiss-cpu
!pip install -U langchain-community tavily-python
!pip gradio_client
!pip install gradio
!pip install langchain_community

# Necessary Imports
import kagglehub
import csv
import pandas as pd
import math
import numpy as np
import os
import getpass
from langchain_core.output_parsers import StrOutputParser
from google.colab import drive

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import create_openai_functions_agent

# Perform necessary imports for creating agents
from langchain import hub # Used to pull predefined prompts from LangChain Hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory  # Used to store chat history in memory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
ERROR: unknown command "gradio_client"
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [2]:
# Setting up Open AI key
os.environ['OPENAI_API_KEY'] = getpass.getpass()


··········


In [3]:
# Loading the data
drive.mount("/content/drive")
path='/content/drive/MyDrive/IK-Company/ProjectUp/Gene/Project/MyData/dataset/prod_small.csv'
df = pd.read_csv(path, index_col=0)

Mounted at /content/drive


In [4]:
df.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000


In [5]:
product_description = []
product_description_len= []
cnt=0
for row in df.iterrows():
  cnt+=1
  if cnt==100:
    break
  product = ''
  title = row[1]['TITLE']
  if type(title) !=float or not math.isnan(title):
    product += "Title" +'\n'+ title+'\n'
  description = row[1]['DESCRIPTION']
  if type(description) !=float or not math.isnan(description):
    product += "Description"+'\n'+description+ '\n'

  content = product or description
  if content:
    product = product.strip()
    product_description.append(product)
    product_description_len.append(len(product))








In [9]:
# Chunking the Data(Product Description)
# See this for more details https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 250,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex= False,
)
documents = text_splitter.create_documents(product_description)
# Create an embedding model using LangChain.
# See https://python.langchain.com/docs/integrations/text_embedding/ for a list of available embedding models on LangChain
embeddings = OpenAIEmbeddings()

# Create a vector store using the created chuns and the embeddings model
vector = FAISS.from_documents(documents, embeddings)

# Create ChatOpenAI object for acting as an LLM.
# See https://python.langchain.com/docs/integrations/chat/openai/
# llm was incorrectly assigned to OpenAIEmbeddings which is for creating embeddings
# Assigning it to ChatOpenAI instead, which is a language model
llm = ChatOpenAI(model='gpt-4',openai_api_key=os.environ['OPENAI_API_KEY'],temperature=0)

# Create a prompt template that gives the model a persona of a customer
# Create a chain for passing a list of Documents to a model.
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""", output_parser = output_parser)

#document_chain = create_stuff_documents_chain(llm,prompt)
document_chain = prompt| llm

# Creating a retriever
# See https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/vectorstore/
retriver = vector.as_retriever()




In [10]:
os.environ['TAVILY_API_KEY'] = getpass.getpass()


··········


In [11]:
# import the 'tool' decorator
from langchain.tools import tool
from langchain.tools.retriever import create_retriever_tool

@tool
def amazon_product_search(query):

  """Search for information about Amazon products.
    For any questions related to Amazon products, this tool must be used."""

  retriver_tool = create_retriever_tool(
        retriver,
        name='amazon_product_search',
        description='Search for information about Amazon products'
    )
  return retriver_tool.invoke(query)
  #return retriver.get_relevant_documents(query)


In [27]:
from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def search_tavily(query):
    """
    Executes a web search using the TavilySearchResults tool.

    Parameters:
        query (str): The search query entered by the user.

    Returns:
        list: A list of search results containing answers, raw content, and images.
    """
    search_tool = TavilySearchResults(
        max_results=5,
        include_answer = True,
        include_raw_content = True,
        include_images = True
    )

    return search_tool.invoke(query)


In [28]:
# hwchase17/react is a prompt template designed for ReAct-style
# conversational agents.
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [29]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [30]:
## Create a list of tools: retriever_tool and search_tool
tools = [amazon_product_search,search_tavily]

In [31]:
react_agent = create_react_agent(
    llm=llm,
    prompt=prompt,
    tools=tools
)

In [32]:
# executes the logical steps we created
agent_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
    )

In [34]:
res = agent_executor.invoke({"input":"What is best clothes on amazon?"})



> Entering new AgentExecutor chain...
I need to search for the best clothes on Amazon.
Action: amazon_product_search
Action Input: best clothesskirts are suitable for pregnant women. We strongly recommend that you customize the dress, select &quot;custom&quot; in the size area, fill in your size, so we will customize this beautiful prom dress for you.

from head to toe (barefoot)<br /><br />1. Originally Designed: All dresses are designed by the most qualified designers, and the most experienced tailors carry out strict quality inspections in our own factories. Every needle and every thread

wear for office, shopping, daily wear in home, college, family, meetings, festive occasions.work " star zari on whole dupatta with jhalar lace. Fancy dupatta party Wear Fancy dupatta for women in chiffon The dupatta is made with Soft Chiffon fabric

for celebratory occasions like weddings and festivals and also wearable as chaniya choli and lancha. Go for high heels and a gold clutch. All for a b

In [35]:
print(res['output'])

The best selling clothes item on Amazon is the "Fruit of the Loom Men's Fleece Full Zip Hoodie".


In [36]:
res = agent_executor.invoke({"input":"What is current weather in Lucknow India?"})



> Entering new AgentExecutor chain...
I need to use a web search tool to find the current weather in Lucknow, India.
Action: search_tavily
Action Input: Current weather in Lucknow India[{'title': 'Lucknow Weather Today (Saturday, Apr 12, 2025)', 'url': 'https://www.indiatoday.in/weather/lucknow-weather-forecast-today', 'content': "Education\nIt's Viral\nScience\nHealth\nAuto\nLaw Today\nEnvironment\nCities\nWeather\nWeb Stories\nHoroscopes\n\n\n\nDownload App\n\nAdvertisement\nAQI\nWeather\nNews / Weather / Lucknow Weather Today\nLucknow Weather Today (Saturday, Apr 12, 2025)\nSat, Apr 12 09:44 PM IST\nClouds\n35\n° c\n\nMin26°\n\nMax39°\n\n\nRise05:45 AM\n\n\nSet06:28 PM\n\n\n\n21Humidity\n\n\n\n1006Pressure\n\n\n\n80.89Lon\n\n\n\n26.83Lat\n\n\nWind\n\n\n\n11.11Speed\n\n\n\n193Deg\n\n\n\n14.11Gust\n\n\nWeekly [...] The minimum temperature in Lucknow today is likely to hover around 26 degrees Celsius, while the maximum temperature might reach 39 degrees Celsius. The mercury level is 

In [37]:
print(res['output'])

The current weather in Lucknow, India is around 35 degrees Celsius with a minimum temperature of 26 degrees Celsius and a maximum of 39 degrees Celsius. The wind speed is around 11.11 with a gust speed of 14.11. The sunrise time is 05:45 AM, while it will set at 06:28 PM.


In [42]:
from logging import PlaceHolder
#building simple UI
import gradio as gr

# Define function for Gradio interface
def chat_with_agent(user_input,session_id):
  """Processes user input and maintains session-based chat history."""
  response = agent_executor.invoke({"input":user_input})

  if isinstance(response, dict) and "output" in response :
    return response['output']
  else:
    return "Error:Unexpected response format"

with gr.Blocks() as app:
  gr.Markdown("🤖 Chat with the Agent")
  gr.Markdown("Enter your query below and get AI powered response with session memory")

  with gr.Row():
    input_box = gr.Textbox(label='Enter youu Query:', placeholder = 'Ask something...')
    output_box = gr.Textbox(label = 'Response', lines=10)

  submit_button = gr.Button('Submit')
  submit_button.click(chat_with_agent,
                      inputs=input_box,
                      outputs=output_box)

  app.launch(debug=True,share=True)






# Create Gradio interface


/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1024: UserWarning: Expected 2 arguments for function <function chat_with_agent at 0x79fe00d21120>, received 1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected at least 2 arguments for function <function chat_with_agent at 0x79fe00d21120>, received 1.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ea7a9fc1141b7850c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/gradio/helpers.py:977: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")




> Entering new AgentExecutor chain...
I need to search for the best clothes on Amazon.
Action: amazon_product_search
Action Input: best clothesskirts are suitable for pregnant women. We strongly recommend that you customize the dress, select &quot;custom&quot; in the size area, fill in your size, so we will customize this beautiful prom dress for you.

from head to toe (barefoot)<br /><br />1. Originally Designed: All dresses are designed by the most qualified designers, and the most experienced tailors carry out strict quality inspections in our own factories. Every needle and every thread

wear for office, shopping, daily wear in home, college, family, meetings, festive occasions.work " star zari on whole dupatta with jhalar lace. Fancy dupatta party Wear Fancy dupatta for women in chiffon The dupatta is made with Soft Chiffon fabric

for celebratory occasions like weddings and festivals and also wearable as chaniya choli and lancha. Go for high heels and a gold clutch. All for a b

/usr/local/lib/python3.11/dist-packages/gradio/helpers.py:977: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")




> Entering new AgentExecutor chain...
The question is asking for the current weather in Shirdi, India. This is not related to Amazon products, so I should use the TavilySearchResults tool.
Action: search_tavily
Action Input: weather in Shirdi India[{'title': 'Shirdi Maharashtra India 15 Day Weather Forecast - Weather Avenue', 'url': 'https://m.weatheravenue.com/en/asia/in/maharashtra/shirdi-weather-15-days.html', 'content': 'Saturday 12 April 2025. Weather Shirdi Saturday 12 April 2025: Sunny Temperature. Sunny. Temperature : 35°C 95°F | 20°C 68°F Temperature Felt : 39°C 103°F', 'score': 0.9534532}, {'title': 'Weather Shirdi in April 2025: Temperature & Climate', 'url': 'https://en.climate-data.org/asia/india/maharashtra/shirdi-717429/t/april-4/', 'content': '31.1 °C\n\n(88) °F\n\n34.7 °C\n\n(94.5) °F\n\n38.5 °C\n\n(101.3) °F\n\n38.6 °C\n\n(101.6) °F\n\n32.7 °C\n\n(90.8) °F\n\n28.4 °C\n\n(83.1) °F\n\n27.7 °C\n\n(81.8) °F\n\n28.9 °C\n\n(84) °F\n\n30.8 °C\n\n(87.4) °F\n\n30 °C\n\n(86) 